In [3]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random
import csv
from keras.callbacks import Callback
from keras.callbacks import CSVLogger
from sklearn.metrics import precision_recall_fscore_support

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms_256HL/Train/'
test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms_256HL/Test/'

num_classes = 7
height = 128
n_fft = 2048
hop_length = 256
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

cnn_model = tf.keras.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Flatten())

# RNN model
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((width, height * channels), input_shape=(height, width, channels)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')),
    tf.keras.layers.BatchNormalization()
])

# Combine CNN and RNN models
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((1, height, width, channels), input_shape=(height, width, channels)),
    tf.keras.layers.TimeDistributed(cnn_model),  # TimeDistributed applies the CNN model to each time step
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')),  # RNN on top of CNN features
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'precision', 'recall', 'f1_score'])

epochs = 185
metrics_data = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_generator.reset()

    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1)

    # Get metrics from history
    accuracy = history.history['accuracy'][0]
    precision = history.history['precision'][0]
    recall = history.history['recall'][0]

    f1_score_tensor = history.history['f1_score'][0]
    f1_score_value = np.mean(f1_score_tensor)

    # Store metrics in a list
    metrics_data.append([accuracy, precision, recall, f1_score_value])

# Save metrics to a CSV file
csv_filename = 'training_metrics.csv'

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Accuracy', 'Precision', 'Recall', 'F1 Score'])
    writer.writerows(metrics_data)

print(f"Training metrics saved to {csv_filename}")

loss, accuracy, precision, recall, f1_score = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1 Score:", f1_score)

model.save('al-emadi_crnn_standardized_unknown_256HL.keras')

Found 3088 images belonging to 7 classes.
Found 768 images belonging to 7 classes.
Epoch 1/185
97/97 ━━━━━━━━━━━━━━━━━━━━ 98s 892ms/step - accuracy: 0.4958 - f1_score: 0.2324 - loss: 1.4878 - precision: 0.7584 - recall: 0.1648
Epoch 2/185
97/97 ━━━━━━━━━━━━━━━━━━━━ 95s 916ms/step - accuracy: 0.7187 - f1_score: 0.5655 - loss: 0.8407 - precision: 0.8851 - recall: 0.5218
Epoch 3/185
97/97 ━━━━━━━━━━━━━━━━━━━━ 93s 917ms/step - accuracy: 0.7936 - f1_score: 0.7068 - loss: 0.6176 - precision: 0.8897 - recall: 0.6754
Epoch 4/185
97/97 ━━━━━━━━━━━━━━━━━━━━ 90s 880ms/step - accuracy: 0.8641 - f1_score: 0.8123 - loss: 0.4215 - precision: 0.9116 - recall: 0.7987
Epoch 5/185
97/97 ━━━━━━━━━━━━━━━━━━━━ 94s 918ms/step - accuracy: 0.8831 - f1_score: 0.8410 - loss: 0.3426 - precision: 0.9232 - recall: 0.8432
Epoch 6/185
97/97 ━━━━━━━━━━━━━━━━━━━━ 90s 883ms/step - accuracy: 0.9003 - f1_score: 0.8803 - loss: 0.3155 - precision: 0.9269 - recall: 0.8727
Epoch 7/185
97/97 ━━━━━━━━━━━━━━━━━━━━ 91s 890ms/step

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_classes = 7
height = 128
n_fft = 2048
hop_length = 256
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

batch_size = 32

model = tf.keras.models.load_model('al-emadi_crnn_standardized_unknown_256HL.keras')

test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms_256HL/Test/'

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

loss, accuracy, precision, recall, f1_score = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1 Score:", f1_score)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Calculate accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

# Print accuracy for each class
print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")

# Print classification report (includes precision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Found 768 images belonging to 7 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 647ms/step - accuracy: 0.9644 - f1_score: 0.6553 - loss: 0.2756 - precision: 0.9668 - recall: 0.9627
Test Loss: 0.20769865810871124
Test Accuracy: 0.96484375
Test Precision: 0.9673202633857727
Test Recall: 0.9635416865348816
Test F1 Score: tf.Tensor(
[0.95238084 1.         0.90476185 1.         0.9900989  0.96901405
 0.9900989 ], shape=(7,), dtype=float32)
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 640ms/step

Confusion Matrix:
[[100   0   2   0   0   2   0]
 [  0  52   0   0   0   0   0]
 [  3   0  95   0   0   6   0]
 [  0   0   0  50   0   0   0]
 [  0   0   0   0  50   0   0]
 [  3   0   9   0   1 344   0]
 [  0   0   0   0   0   1  50]]

Accuracy for each class:
Bebop: 0.9615
Crazyflie: 1.0000
Mambo: 0.9135
Matrice100: 1.0000
Mavic: 1.0000
NonDrone: 0.9636
Phantom: 0.9804

Classification Report:
              precision    recall  f1-score   support

       Bebop       0.94      0.96      0.95       104
   Crazyflie       1.00      1.00      1.00        52
      

In [52]:
import os
import numpy as np
from scipy.stats import entropy
from tensorflow.keras.preprocessing import image

model = tf.keras.models.load_model('/content/al-emadi_crnn_standardized_unknown_256HL.keras')
classes = ['Bebop', 'Crazyflie', 'Mambo', 'Matrice100', 'Mavic', 'NonDrone', 'Phantom']

known_counter = 0
unknown_counter = 0

def load_spectrogram(file_path):
    # Load and preprocess the image
    num_classes = 7
    height = 128
    n_fft = 2048
    hop_length = 256
    sr = 16000
    T = 1

    hop_size_sec = hop_length / sr
    audio_length_samples = T * sr

    width = 1 + int((audio_length_samples - n_fft) / hop_length)

    channels = 3

    img = image.load_img(file_path, target_size=(height, width))  # Assuming input size expected by your model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    #img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array

# Function to classify a single spectrogram
def classify_spectrogram(spectrogram, entropy_threshold):
    global known_counter
    global unknown_counter

    probabilities = model.predict(spectrogram)[0]  # Get class probabilities from the model
    ent = entropy(probabilities, base=2)  # Calculate entropy of the probability distribution

    if ent > entropy_threshold:
        unknown_counter+=1
        return "UnknownDrone"
    else:
        known_counter+=1
        max_index = np.argmax(probabilities)  # Get the index of the maximum probability
        return classes[max_index]  # Return the corresponding drone class

# Directory containing spectrogram files
unknown_spectrogram_dir = '/content/drive/MyDrive/UnknownDrone_256HL/'
spectrogram_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms_256HL/Test/Bebop/'

# VALUE UP = UNKNOWN DRONE DECREASE | VALUE DOWN = UNKNOWN DRONE INCREASE
entropy_threshold =  0.12155  # Adjust this threshold based on your model and validation results

# List all files in the directory
spectrogram_files = os.listdir(spectrogram_dir)

# Process each spectrogram file
for filename in spectrogram_files:
    if filename.endswith('.png'):  # Assuming spectrograms are saved as PNG files
        # Load the spectrogram (you need to implement this part based on your data loading method)
        spectrogram = load_spectrogram(os.path.join(spectrogram_dir, filename))

        # Classify the spectrogram
        predicted_class = classify_spectrogram(spectrogram, entropy_threshold)

        # Print the filename and predicted class
        print(f"{filename}: {predicted_class}")

print("Unknown Drone Count:", unknown_counter)
print("Known Drone Count:", known_counter)
print("Accuracy: ", (known_counter/(known_counter+unknown_counter)*100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
B_S2_D1_086-bebop_000__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
mixed_19-bebop_000__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
B_S2_D1_070-bebop_004__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
mixed_14-bebop_000__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
mixed_8-bebop_003__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
mixed_0-bebop_002__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
B_S2_D1_101-bebop_004__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
mixed_18-bebop_002__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
mixed_5-bebop_002__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
B_S2_D1_070-bebop_002__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
B_S2_D1_101-bebop_000__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
mixed_10-bebop_001__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
B_S2_D1_086-bebop_003__mel.png: Bebop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
